In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/questions/questions.csv


In [2]:
import os
import numpy as np
import pandas as pd
import random as rnd
import tensorflow as tf

# Set random seeds
rnd.seed(34)

2025-09-03 07:19:32.304222: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756883972.652187      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756883972.751695      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
data = pd.read_csv("/kaggle/input/questions/questions.csv")
N = len(data)
print('Number of question pairs: ', N)
data.head()

Number of question pairs:  404351


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
N_train = 300000
N_test = 10240
data_train = data[:N_train]
data_test = data[N_train:N_train + N_test]
print("Train set:", len(data_train), "Test set:", len(data_test))
del (data)  # remove to free memory

Train set: 300000 Test set: 10240


In [5]:
td_index = data_train['is_duplicate'] == 1
td_index = [i for i, x in enumerate(td_index) if x]
print('Number of duplicate questions: ', len(td_index))
print('Indexes of first ten duplicate questions:', td_index[:10])

Number of duplicate questions:  111486
Indexes of first ten duplicate questions: [5, 7, 11, 12, 13, 15, 16, 18, 20, 29]


In [6]:
print(data_train['question1'][5])
print(data_train['question2'][5])
print('is_duplicate: ', data_train['is_duplicate'][5])

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?
is_duplicate:  1


In [7]:
Q1_train = np.array(data_train['question1'][td_index])
Q2_train = np.array(data_train['question2'][td_index])

Q1_test = np.array(data_test['question1'])
Q2_test = np.array(data_test['question2'])
y_test  = np.array(data_test['is_duplicate'])

In [8]:
print('TRAINING QUESTIONS:\n')
print('Question 1: ', Q1_train[0])
print('Question 2: ', Q2_train[0], '\n')
print('Question 1: ', Q1_train[5])
print('Question 2: ', Q2_train[5], '\n')

print('TESTING QUESTIONS:\n')
print('Question 1: ', Q1_test[0])
print('Question 2: ', Q2_test[0], '\n')
print('is_duplicate =', y_test[0], '\n')

TRAINING QUESTIONS:

Question 1:  Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
Question 2:  I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me? 

Question 1:  What would a Trump presidency mean for current international master’s students on an F1 visa?
Question 2:  How will a Trump presidency affect the students presently in US or planning to study in US? 

TESTING QUESTIONS:

Question 1:  How do I prepare for interviews for cse?
Question 2:  What is the best way to prepare for cse? 

is_duplicate = 0 



In [9]:
cut_off = int(len(Q1_train) * 0.8)
train_Q1, train_Q2 = Q1_train[:cut_off], Q2_train[:cut_off]
val_Q1, val_Q2 = Q1_train[cut_off:], Q2_train[cut_off:]
print('Number of duplicate questions: ', len(Q1_train))
print("The length of the training set is:  ", len(train_Q1))
print("The length of the validation set is: ", len(val_Q1))

Number of duplicate questions:  111486
The length of the training set is:   89188
The length of the validation set is:  22298


In [10]:
train_Q1 = train_Q1.reshape(-1, 1)
train_Q2 = train_Q2.reshape(-1, 1)
val_Q1 = val_Q1.reshape(-1, 1)
val_Q2 = val_Q2.reshape(-1, 1)

In [11]:
tf.random.set_seed(0)
text_vectorization = tf.keras.layers.TextVectorization(output_mode='int',split='whitespace', standardize='strip_punctuation')
text_vectorization.adapt(np.concatenate((Q1_train,Q2_train)))

I0000 00:00:1756884054.444485      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1756884054.445197      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [12]:
print(f'Vocabulary size: {text_vectorization.vocabulary_size()}')

Vocabulary size: 36224


In [13]:
print('first question in the train set:\n')
print(Q1_train[0], '\n') 
print('encoded version:')
print(text_vectorization(Q1_train[0]),'\n')

print('first question in the test set:\n')
print(Q1_test[0], '\n')
print('encoded version:')
print(text_vectorization(Q1_test[0]) )

first question in the train set:

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me? 

encoded version:
tf.Tensor(
[ 6984     6   178    10  8988  2442 35393   761    13  6636 28205    31
    28   483    45    98], shape=(16,), dtype=int64) 

first question in the test set:

How do I prepare for interviews for cse? 

encoded version:
tf.Tensor([    4     8     6   160    17  2079    17 11775], shape=(8,), dtype=int64)


In [14]:

def Siamese(text_vectorizer, vocab_size=36224, d_feature=128):

    branch = tf.keras.models.Sequential(name='sequential') 
    branch.add(text_vectorizer)
    branch.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=d_feature, name="embedding"))
    branch.add(tf.keras.layers.LSTM(d_feature, return_sequences=True, name="LSTM"))
    branch.add(tf.keras.layers.GlobalAveragePooling1D(name="mean"))
    branch.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x), name="out"))
    
    # Define both inputs
    input1 = tf.keras.layers.Input(shape=(1,), dtype=tf.string, name='input_1')
    input2 = tf.keras.layers.Input(shape=(1,), dtype=tf.string, name='input_2')
    
    # Apply branch
    branch1 = branch(input1)
    branch2 = branch(input2)
    
    conc = tf.keras.layers.Concatenate(axis=1, name='conc_1_2')([branch1, branch2])
    
    return tf.keras.models.Model(inputs=[input1, input2], outputs=conc, name="SiameseModel")

In [15]:
model = Siamese(text_vectorization, vocab_size=text_vectorization.vocabulary_size())
model.build(input_shape=None)
model.summary()
model.get_layer(name='sequential').summary()

Model: "SiameseModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_1             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_2             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 128)       │  4,768,256 │ input_1[0][0],    │
│ (Sequential)        │                   │            │ input_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conc_1_2            │ (None, 256)       │          0 │ sequential[0][0], │
│ (Concatenate)       │                   │            │ sequential[1][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,768,256 (18.19 MB)

 Trainable params: 4,768,256 (18.19 MB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, None)           │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 128)      │     4,636,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ LSTM (LSTM)                     │ (None, None, 128)      │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mean (GlobalAveragePooling1D)   │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ out (Lambda)                    │ (None, 128)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,768,256 (18.19 MB)

 Trainable params: 4,768,256 (18.19 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
def TripletLossFn(v1, v2, margin=0.25):
    """Custom Triplet Loss with hard negative mining and debug prints."""

    scores = tf.linalg.matmul(v1, v2, transpose_b=True)
    positive_scores = tf.linalg.diag_part(scores)

    mask = tf.eye(tf.shape(scores)[0], dtype=scores.dtype) 
    hardest_negative_scores = tf.reduce_max(scores - (mask * 1e9), axis=1)

    loss = tf.maximum(0.0, margin - positive_scores + hardest_negative_scores)

    return tf.reduce_mean(loss)

In [17]:
v1 = np.array([[0.26726124, 0.53452248, 0.80178373],[0.5178918 , 0.57543534, 0.63297887]])
v2 = np.array([[ 0.26726124,  0.53452248,  0.80178373],[-0.5178918 , -0.57543534, -0.63297887]])
v1 = v1.astype(np.float32)
v2 = v2.astype(np.float32)

print("Triplet Loss:", TripletLossFn(v1,v2).numpy())

Triplet Loss: 1.1017538


In [18]:
batch_size = 64 

train_dataset = tf.data.Dataset.from_tensor_slices(((train_Q1, train_Q2),tf.constant([1]*len(train_Q1))))
train_dataset = train_dataset.batch(batch_size) 

val_dataset = tf.data.Dataset.from_tensor_slices(((val_Q1, val_Q2),tf.constant([1]*len(val_Q1))))
val_dataset = val_dataset.batch(batch_size)

In [19]:
def TripletLoss(labels, out, margin=0.25):
    _, embedding_size = out.shape 
    v1 = out[:,:int(embedding_size/2)] 
    v2 = out[:,int(embedding_size/2):] 
    return TripletLossFn(v1, v2, margin=margin)

In [20]:
from tensorflow.keras.optimizers import Adam

In [21]:
def train_model(model, train_dataset, val_dataset, lr=0.001, epochs=10):
    """
    Trains the Siamese model.
    
    Args:
        model (tf.keras.Model): The Siamese model to train.
        train_dataset (tf.data.Dataset): Training data iterator.
        val_dataset (tf.data.Dataset): Validation data iterator.
        lr (float): The learning rate for the optimizer.
        epochs (int): The number of epochs to train for.
        
    Returns:
        tf.keras.callbacks.History: The history object from the training process.
    """
    
    optimizer = Adam(learning_rate=lr)

    model.compile(loss=TripletLoss, optimizer=optimizer)

    history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=val_dataset
    )
    
    return history

In [22]:
train_epochs = 25 
learning_rate = 0.001

# Train the model
history = train_model(model, train_dataset, val_dataset, learning_rate, train_epochs)

print("\nTraining complete!")


Epoch 1/25


I0000 00:00:1756884094.586050      96 cuda_dnn.cc:529] Loaded cuDNN version 90300


1394/1394 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - loss: 0.2484 - val_loss: 0.2429
Epoch 2/25
1394/1394 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.2424 - val_loss: 0.2424
Epoch 3/25
1394/1394 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.2417 - val_loss: 0.2422
Epoch 4/25
1394/1394 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.2421 - val_loss: 0.2430
Epoch 5/25
1394/1394 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.2416 - val_loss: 0.2421
Epoch 6/25
1394/1394 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.2410 - val_loss: 0.2418
Epoch 7/25
1394/1394 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.2406 - val_loss: 0.2415
Epoch 8/25
1394/1394 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.2403 - val_loss: 0.2415
Epoch 9/25
1394/1394 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.2402 - val_loss: 0.2417
Epoch 10/25
1394/1394 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.2400 - val_loss: 0.2417
Epoch 11/25
1394/1394 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - loss: 0.2399 - val_loss: 0.2416
Epoch 12/25
1394/1394 ━━━━━━━

In [23]:
def classify(Q1_test, Q2_test, y_test, threshold, model, batch_size=64):
    """
    Classifies question pairs and evaluates the model's accuracy.
    """
    num_samples = len(Q1_test)
    all_preds = []

    # Process the data in batches
    for i in range(0, num_samples, batch_size):
       
        q1_batch = Q1_test[i : i + batch_size].reshape(-1, 1)
        q2_batch = Q2_test[i : i + batch_size].reshape(-1, 1)

        q1_tensor = tf.convert_to_tensor(q1_batch, dtype=tf.string)
        q2_tensor = tf.convert_to_tensor(q2_batch, dtype=tf.string)
        
    
        pred = model.predict((q1_tensor, q2_tensor), verbose=0)
        

        embedding_size = pred.shape[1]
        v1 = pred[:, :embedding_size//2]
        v2 = pred[:, embedding_size//2:]

        d = tf.reduce_sum(v1 * v2, axis=1)
        
        batch_preds = (d > threshold).numpy().astype(int)
        all_preds.extend(batch_preds)

    y_test = y_test.astype(int)
    all_preds = np.array(all_preds)
    
    accuracy = np.mean(all_preds == y_test)
    cm = tf.math.confusion_matrix(labels=y_test, predictions=all_preds)

    return accuracy, cm

In [27]:
Q1_test = Q1_test.astype(str)
Q2_test = Q2_test.astype(str)
y_test = y_test.astype(int)

classification_threshold = 0.5
accuracy, confusion_matrix = classify(Q1_test, Q2_test, y_test, 
                                      classification_threshold, model)

print(f"Accuracy on the test set: {accuracy:.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix.numpy())

Accuracy on the test set: 0.6265

Confusion Matrix:
[[6346   36]
 [3789   69]]


In [30]:
def predict_similarity(question1, question2, model):
    """
    Predicts the similarity between two questions using the trained Siamese model.
    """
    q1_arr = np.array([question1]).reshape(-1, 1)
    q2_arr = np.array([question2]).reshape(-1, 1)
    
    q1_tensor = tf.convert_to_tensor(q1_arr, dtype=tf.string)
    q2_tensor = tf.convert_to_tensor(q2_arr, dtype=tf.string)
    
    pred = model.predict((q1_tensor, q2_tensor))
    
    embedding_size = pred.shape[1] // 2
    v1 = pred[:, :embedding_size]
    v2 = pred[:, embedding_size:]
    
    similarity = np.dot(v1, v2.T)[0, 0]
    
    return similarity

In [43]:
# A pair of questions that are duplicates
q_duplicate_1 = "How old are you?"
q_duplicate_2 = "What is your name?"

# A pair of questions that are not duplicates
q_different_1 = "Where do you live?"
q_different_2 = "What do you eat?"

# Calculate similarity
similar_score = predict_similarity(q_duplicate_1, q_duplicate_2, model)
different_score = predict_similarity(q_different_1, q_different_2, model)

print(f"Similarity for duplicate questions: {similar_score:.4f}")
print(f"Similarity for different questions: {different_score:.4f}")

threshold = 0.55
print(f"\nAre the first two questions duplicates? {'Yes' if similar_score > threshold else 'No'}")
print(F"\nAre the second pair of questions duplicate? {'Yes' if different_score > threshold else 'No'}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Similarity for duplicate questions: 0.5946
Similarity for different questions: 0.4465

Are the first two questions duplicates? Yes

Are the second pair of questions duplicate? No
